<a href="https://colab.research.google.com/github/Alphambarushimana/Sustainability/blob/main/M2_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [382]:
!pip install -qq -U gensim
!pip install -qq pyLDAvis
!pip install tweet-preprocessor -q
!pip install --upgrade gensim
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [381]:
import pandas as pd
import numpy as np
import preprocessor as prepro 
import tqdm
import gensim 

# spacy for quick language prepro
import spacy 
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

# viz
import altair as alt 

# explainability
import eli5
from eli5.lime import TextExplainer

# topic modeling
from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA

# import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()


# network analysis
import pandas as pd
import networkx as nx
from community import community_louvain
import altair as alt

# Natural Language Processing (NLP)

## Data preparation

In [291]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

In [292]:
data = pd.read_csv("https://raw.githubusercontent.com/Alphambarushimana/Sustainability/main/scopuswithlabels.csv")

In [293]:
data

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Affiliations,Authors with affiliations,Abstract,Author Keywords,Index Keywords,Document Type,Publication Stage,Open Access,Source,EID
0,"Chapwanya M., Lubuma J., Terefe Y., Tsanou B.",26026788700;6602950766;54783401400;25823549000;,Analysis of War and Conflict Effect on the Tra...,2022,Bulletin of Mathematical Biology,84,12,136,NaN,NaN,...,Department of Mathematics and Applied Mathemat...,"Chapwanya, M., Department of Mathematics and A...",The tenth Ebola outbreak in the Democratic Rep...,Basic reproduction number; Conflict dynamics; ...,basic reproduction number; biological model; D...,Article,Final,NaN,Scopus,2-s2.0-85140065004
1,"Piat M., Wainwright M., Rivest M.-P., Sofouli ...",6604070039;35090631400;56871264100;18635052000...,The impacts of implementing recovery innovatio...,2022,International Journal of Mental Health Systems,16,1,49,NaN,NaN,...,"Department of Psychiatry, McGill University an...","Piat, M., Department of Psychiatry, McGill Uni...",Background: Implementing mental health recover...,Conceptual framework; Guidelines; Impact; Impl...,adult; article; conceptual framework; content ...,Article,Final,NaN,Scopus,2-s2.0-85139748940
2,"Velayati F., Ayatollahi H., Hemmat M., Dehghan R.",57219486899;55991128900;57170342800;57365458900;,The 4P telehealth business framework for Iran,2022,BMC Medical Informatics and Decision Making,22,1,266,NaN,NaN,...,"Telemedicine Research Center, National Researc...","Velayati, F., Telemedicine Research Center, Na...",Background: Telehealth services can utilize va...,Business; Health service marketing; Telehealth...,adult; article; comparative effectiveness; Del...,Article,Final,NaN,Scopus,2-s2.0-85139435002
3,"Pastor A.V., Nunes J.P., Ciampalini R., Bahri ...",55927118200;7102878032;6506995140;7004276837;5...,ScenaLand: a simple methodology for developing...,2022,Mitigation and Adaptation Strategies for Globa...,27,8,52,NaN,NaN,...,"LISAH, University Montpellier, INRAE, IRD, Ins...","Pastor, A.V., LISAH, University Montpellier, I...",Scenarios serve science by testing the sensiti...,Experts; Land degradation; Land use and manage...,agricultural soil; environmental protection; e...,Article,Final,NaN,Scopus,2-s2.0-85138219727
4,"Abdullah H.M., Dakurah T., Akoto H., Abaidoo B...",57887451300;12140148900;57213456448;5623694480...,Predictors of recurrence of chronic subdural h...,2022,BMC Neurology,22,1,346,NaN,NaN,...,"Department of Surgery, Korle Bu Teaching Hospi...","Abdullah, H.M., Department of Surgery, Korle B...",Background: Chronic subdural haematoma (CSDH) ...,Association; Chronic; Haematoma; Predictors; R...,anticoagulant agent; antithrombocytic agent; w...,Article,Final,NaN,Scopus,2-s2.0-85137878264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,"Armeanu D.S., Vintilă G., Gherghina S.C.",26434149400;26434536500;56046530600;,Empirical study towards the drivers of sustain...,2018,Sustainability (Switzerland),10,1,4,NaN,NaN,...,"Department of Finance, The Bucharest Universit...","Armeanu, D.S., Department of Finance, The Buch...",This study aims at empirically investigating t...,Economic growth; EU-28; Fixed and random effec...,economic growth; employment; European Union; h...,Article,Final,NaN,Scopus,2-s2.0-85038843701
1630,"Abaunza F., Hameri A.-P., Niemi T.",56060899300;7003735163;57215042589;,EEUI: a new measure to monitor and manage ener...,2018,International Journal of Productivity and Perf...,67,1,NaN,111,127,...,"Faculty of Business and Economics, University ...","Abaunza, F., Faculty of Business and Economics...",Purpose: Data centers (DCs) are similar to tra...,Data centres; Energy efficiency; Energy propor...,NaN,Article,Final,NaN,Scopus,2-s2.0-85038831501
1631,"Zhang Y., Lin Z.",57840767400;54894812400;,Predicting the helpfulness of online product r...,2018,Electronic Commerce Research and Applications,27,NaN,NaN,1,10,...,"School of

In [294]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634 entries, 0 to 1633
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Authors                    1634 non-null   object 
 1   Author(s) ID               1634 non-null   object 
 2   Title                      1634 non-null   object 
 3   Year                       1634 non-null   int64  
 4   Source title               1634 non-null   object 
 5   Volume                     1219 non-null   object 
 6   Issue                      930 non-null    object 
 7   Art. No.                   480 non-null    object 
 8   Page start                 1046 non-null   object 
 9   Page end                   1026 non-null   object 
 10  Page count                 47 non-null     float64
 11  Cited by                   1005 non-null   float64
 12  DOI                        1428 non-null   object 
 13  Link                       1634 non-null   objec

In [295]:
data = data[['Authors', 'Author(s) ID','Title', 'Abstract','Year', 'Source title', 'Author Keywords']]

## Topic modelling




In [296]:
# creating a new column.
data ['text'] = '. ' + data['Abstract']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [297]:
data['text_clean'] = data['text'].map(lambda t: prepro.clean(t))
data['text_clean'] = data['text_clean'].str.replace('#','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [298]:
# run progress bar and clean up using spacy but without some heavy parts of the pipeline
clean_text = []

pbar = tqdm.tqdm(total=len(data['text_clean']),position=0, leave=True)

for text in nlp.pipe(data['text_clean'], disable=["tagger", "parser", "ner"]):

  txt = [token.lemma_.lower() for token in text 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]

  clean_text.append(" ".join(txt))

  pbar.update(1)

100%|██████████| 1634/1634 [03:11<00:00,  8.55it/s] 
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
 99%|█████████▉| 1619/1634 [00:25<00:00, 101.83it/s]

In [299]:
# write everything into one function that can be re-used later
def text_prepro(texts):
  """
  takes in a pandas series (1 column of a DF)
  removes twitter stuff
  lowercases, normalizes text
  """
  texts_clean = texts.map(lambda t: prepro.clean(t))
  texts_clean = texts_clean.str.replace('#','')

  clean_container = []

  pbar = tqdm.tqdm(total=len(texts_clean),position=0, leave=True)

  for text in nlp.pipe(texts_clean, disable=["tagger", "parser", "ner"]):

    txt = [token.lemma_.lower() for token in text 
          if token.is_alpha 
          and not token.is_stop 
          and not token.is_punct]

    clean_container.append(" ".join(txt))
    pbar.update(1)
  
  return clean_container

In [300]:
# apply all prepro-pipeline to texts
data['text_clean'] = text_prepro(data['text'])

100%|██████████| 1634/1634 [00:24<00:00, 66.97it/s] 
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [301]:
data

,Authors,Author(s) ID,Title,Abstract,Year,Source title,Author Keywords,text,text_clean
0,"Chapwanya M., Lubuma J., Terefe Y., Tsanou B.",26026788700;6602950766;54783401400;25823549000;,Analysis of War and Conflict Effect on the Tra...,The tenth Ebola outbreak in the Democratic Rep...,2022,Bulletin of Mathematical Biology,Basic reproduction number; Conflict dynamics; ...,. The tenth Ebola outbreak in the Democratic R...,tenth ebola outbreak democratic republic congo...
1,"Piat M., Wainwright M., Rivest M.-P., Sofouli ...",6604070039;35090631400;56871264100;18635052000...,The impacts of implementing recovery innovatio...,Background: Implementing mental health recover...,2022,International Journal of Mental Health Systems,Conceptual framework; Guidelines; Impact; Impl...,. Background: Implementing mental health recov...,background implementing mental health recovery...
2,"Velayati F., Ayatollahi H., Hemmat M., Dehghan R.",57219486899;55991128900;57170342800;57365458900;,The 4P telehealth business framework for Iran,Background: Telehealth services can utilize va...,2022,BMC Medical Informatics and Decision Making,Business; Health service marketing; Telehealth...,. Background: Telehealth services can utilize ...,background telehealth services utilize informa...
3,"Pastor A.V., Nunes J.P., Ciampalini R., Bahri ...",55927118200;7102878032;6506995140;7004276837;5...,ScenaLand: a simple methodology for developing...,Scenarios serve science by testing the sensiti...,2022,Mitigation and Adaptation Strategies for Globa...,Experts; Land degradation; Land use and manage...,. Scenarios serve science by testing the sensi...,scenarios serve science testing sensitivity sy...
4,"Abdullah H.M., Dakurah T., Akoto H., Abaidoo B...",57887451300;12140148900;57213456448;5623694480...,Predictors of recurrence of chronic subdural h...,Background: Chronic subdural haematoma (CSDH) ...,2022,BMC Neurology,Association; Chronic; Haematoma; Predictors; R...,. Background: Chronic subdural haematoma (CSDH...,background chronic subdural haematoma csdh com...
...,...,...,...,...,...,...,...,...,...
1629,"Armeanu D.S., Vintilă G., Gherghina S.C.",26434149400;26434536500;56046530600;,Empirical study towards the drivers of sustain...,This study aims at empirically investigating t...,2018,Sustainability (Switzerland),Economic growth; EU-28; Fixed and random effec...,. This study aims at empirically investigating...,study aims empirically investigating drivers s...
1630,"Abaunza F., Hameri A.-P., Niemi T.",56060899300;7003735163;57215042589;,EEUI: a new measure to monitor and manage ener...,Purpose: Data centers (DCs) are similar to tra...,2018,International Journal of Productivity and Perf...,Data centres; Energy efficiency; Energy propor...,. Purpose: Data centers (DCs) are similar to t...,purpose data centers dcs similar traditional f...
1631,"Zhang Y., Lin Z.",57840767400;54894812400;,Predicting the helpfulness of online product r...,Identifying helpful reviews from massive revie...,2018,Electronic Commerce Research and Applications,Multilingual reviews; Prediction; Product revi...,. Identifying helpful reviews from massive rev...,identifying helpful reviews massive review dat...
1632,"Greening H., Swann R., St. Pé K., Testroet-Ber...",6602993313;36025859800;15038078600;57196705444...,Local implementation of a national program: Th...,The US Environmental Protection Agency's Natio...,2018,Marine Policy,Education; Environmental restoration; Governan...,. The US Environmental Protection Agency's Nat...,environmental protection agency national estua...


In [302]:
# preprocess texts (we need tokens)
tokens = []

for summary in nlp.pipe(data['text_clean'], disable=["ner"]):
  proj_tok = [token.lemma_.lower() for token in summary 
              if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] 
              and not token.is_stop
              and not token.is_punct] 
  tokens.append(proj_tok)

In [303]:
data['tokens'] = tokens

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [304]:
vectorizer1 = TfidfVectorizer() 
vectors1 = vectorizer.fit_transform(data['text_clean'])

In [305]:
# Create a Dictionary from the articles: dictionary
dictionary3 = Dictionary(data['tokens'])
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary3.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

In [306]:
# construct corpus using this dictionary
corpus_tfidf1 = [dictionary3.doc2bow(doc) for doc in data['tokens']]

In [307]:
corpus_tfidf1

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 2),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 5),
  (10, 1),
  (11, 1),
  (12, 2),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 4),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 2),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 8),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2),
  (38, 2),
  (39, 2),
  (40, 4),
  (41, 1),
  (42, 1),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1)],
 [(2, 1),
  (20, 1),
  (21, 3),
  (25, 8),
  (27, 1),
  (48, 2),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 2),
  (68, 4),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 3),
  (74, 4),
  (75, 6),
  (76, 2),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 2),
  (83, 2),
  (84, 1),
  (85, 1)

In [308]:
import gensim

In [309]:
lda_model_tfidf1 = gensim.models.LdaMulticore(corpus_tfidf1, num_topics=3, id2word=dictionary3, passes=2, workers=4)

In [310]:
lda_display3 = pyLDAvis.gensim_models.prepare(lda_model_tfidf1, corpus_tfidf1, dictionary3)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [311]:
pyLDAvis.display(lda_display3)

## EDA - Dynamic effects (over time)

Let's analyze the topics among different years with some visualization. Before, we need to get the correct data frame.

## Clustering

We are using the LDA algorythm for clustering



In [313]:
data1 = data[['Abstract', 'Author Keywords']]

In [314]:
data1

,Abstract,Author Keywords
0,The tenth Ebola outbreak in the Democratic Rep...,Basic reproduction number; Conflict dynamics; ...
1,Background: Implementing mental health recover...,Conceptual framework; Guidelines; Impact; Impl...
2,Background: Telehealth services can utilize va...,Business; Health service marketing; Telehealth...
3,Scenarios serve science by testing the sensiti...,Experts; Land degradation; Land use and manage...
4,Background: Chronic subdural haematoma (CSDH) ...,Association; Chronic; Haematoma; Predictors; R...
...,...,...
1629,This study aims at empirically investigating t...,Economic growth; EU-28; Fixed and random effec...
1630,Purpose: Data centers (DCs) are similar to tra...,Data centres; Energy efficiency; Energy propor...
1631,Identifying helpful reviews from massive revie...,Multilingual reviews; Prediction; Product revi...
1632,The US Environmental Protection Agency's Natio...,Education; Environmental restoration; Governan...


In [315]:
data1['Author Keywords']= data1['Author Keywords'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [316]:
# preprocess texts (we need tokens)
tokens = []

for summary in nlp.pipe(data1['Author Keywords'], disable=["ner"]):
  proj_tok = [token.lemma_.lower() for token in summary 
              if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] 
              and not token.is_stop
              and not token.is_punct] 
  tokens.append(proj_tok)

In [317]:
data1['tokens'] = tokens

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [318]:
vectorizer = TfidfVectorizer() 
vectors = vectorizer.fit_transform(data1['Author Keywords'])

In [319]:
# Create a Dictionary from the articles: dictionary
dictionary2 = Dictionary(data1['tokens'])
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary2.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

In [320]:
# construct corpus using this dictionary
corpus_tfidf = [dictionary2.doc2bow(doc) for doc in data1['tokens']]

In [321]:
corpus_tfidf

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1)],
 [(5, 1), (14, 1), (15, 1), (16, 1), (17, 1)],
 [(18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)],
 [(24, 1)],
 [(14, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)],
 [(31, 2), (32, 2), (33, 2), (34, 1)],
 [(35, 1), (36, 2)],
 [(34, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1)],
 [(0, 1), (5, 1), (12, 1), (27, 1), (44, 1)],
 [(15, 1), (43, 1), (45, 1), (46, 1), (47, 1)],
 [(48, 1)],
 [(48, 1)],
 [(49, 1), (50, 1)],
 [(48, 1)],
 [(11, 1),
  (12, 1),
  (51, 1),
  (52, 2),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1)],
 [(48, 1)],
 [(11, 1), (20, 1), (60, 1), (61, 1), (62, 1)],
 [(20, 1), (31, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)],
 [(11, 1), (20, 1), (37, 1), (69, 1), (70, 1), (71, 1), (72, 1)],
 [(73, 1), (74, 1), (75, 2), (76, 1), (77, 1)],
 [(34, 1), (49, 1), (

In [323]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=3, id2word=dictionary2, passes=2, workers=4)

In [324]:
lda_display2 = pyLDAvis.gensim_models.prepare(lda_model_tfidf, corpus_tfidf, dictionary2)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [325]:
pyLDAvis.display(lda_display2)

## Model trainig

In [326]:
transf_corpus = lda_model_tfidf.get_document_topics(corpus_tfidf)

In [327]:
l=[lda_model_tfidf.get_document_topics(item) for item in corpus_tfidf]

In [328]:
l

[[(0, 0.0803024), (1, 0.12514189), (2, 0.79455566)],
 [(0, 0.58348995), (1, 0.037795108), (2, 0.37871495)],
 [(0, 0.55001533), (1, 0.061876006), (2, 0.38810867)],
 [(0, 0.89795744), (1, 0.05110055), (2, 0.050941993)],
 [(0, 0.66059834), (1, 0.16824622), (2, 0.17115544)],
 [(0, 0.058588345), (1, 0.04565219), (2, 0.89575946)],
 [(0, 0.9106711), (1, 0.046562675), (2, 0.042766202)],
 [(0, 0.09083552), (1, 0.80538404), (2, 0.10378048)],
 [(0, 0.037423234), (1, 0.92511344), (2, 0.037463345)],
 [(0, 0.8782931), (1, 0.058443073), (2, 0.06326387)],
 [(0, 0.063368216), (1, 0.06950245), (2, 0.8671293)],
 [(0, 0.6642805), (1, 0.16770402), (2, 0.16801548)],
 [(0, 0.6642806), (1, 0.16770403), (2, 0.1680154)],
 [(0, 0.74367136), (1, 0.13364308), (2, 0.12268557)],
 [(0, 0.6642803), (1, 0.16770408), (2, 0.16801558)],
 [(0, 0.03227085), (1, 0.030171541), (2, 0.9375576)],
 [(0, 0.6642804), (1, 0.16770403), (2, 0.16801558)],
 [(0, 0.06277868), (1, 0.07622894), (2, 0.86099243)],
 [(0, 0.91590714), (1, 0.04

In [329]:
df = pd.DataFrame(l)

In [330]:
df.head()

,0,1,2
0,"(0, 0.0803024)","(1, 0.12514189)","(2, 0.79455566)"
1,"(0, 0.58348995)","(1, 0.037795108)","(2, 0.37871495)"
2,"(0, 0.55001533)","(1, 0.061876006)","(2, 0.38810867)"
3,"(0, 0.89795744)","(1, 0.05110055)","(2, 0.050941993)"
4,"(0, 0.66059834)","(1, 0.16824622)","(2, 0.17115544)"


In [331]:
sorted([('abc', 121),('abc', 231),('abc', 148), ('abc',221)], 
       key=lambda x: x[1], reverse=True)

[('abc', 231), ('abc', 221), ('abc', 148), ('abc', 121)]

In [332]:
sorted(l[1], key=lambda x: x[1], reverse=True)[0][0]

0

In [333]:
labels = []

for blah in l:
  ll = sorted(blah, key=lambda x: x[1], reverse=True)[0][0]
  labels.append(ll)



In [334]:
labels

[2,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 1,
 1,
 0,
 0,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 2,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 1,
 2,
 2,
 2,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 1,
 2,
 0,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 2,
 1,
 0,
 2,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 0,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 1,


In [335]:
labels = pd.DataFrame(labels)

In [336]:
labels.head()

,0
0,2
1,0
2,0
3,0
4,0


In [337]:
data1['labels'] = labels

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [338]:
data1

,Abstract,Author Keywords,tokens,labels
0,The tenth Ebola outbreak in the Democratic Rep...,Basic reproduction number; Conflict dynamics; ...,"[basic, reproduction, number, conflict, dynami...",2
1,Background: Implementing mental health recover...,Conceptual framework; Guidelines; Impact; Impl...,"[conceptual, framework, guideline, impact, imp...",0
2,Background: Telehealth services can utilize va...,Business; Health service marketing; Telehealth...,"[business, health, service, marketing, telehea...",0
3,Scenarios serve science by testing the sensiti...,Experts; Land degradation; Land use and manage...,"[expert, land, degradation, land, use, managem...",0
4,Background: Chronic subdural haematoma (CSDH) ...,Association; Chronic; Haematoma; Predictors; R...,"[association, chronic, haematoma, predictors, ...",0
...,...,...,...,...
1629,This study aims at empirically investigating t...,Economic growth; EU-28; Fixed and random effec...,"[economic, growth, fixed, random, effect, mode...",2
1630,Purpose: Data centers (DCs) are similar to tra...,Data centres; Energy efficiency; Energy propor...,"[datum, centre, energy, efficiency, energy, pr...",0
1631,Identifying helpful reviews from massive revie...,Multilingual reviews; Prediction; Product revi...,"[multilingual, review, prediction, product, re...",0
1632,The US Environmental Protection Agency's Natio...,Education; Environmental restoration; Governan...,"[education, environmental, restoration, govern...",0


In [339]:
data1.labels.value_counts().reset_index()

,index,labels
0,0,696
1,2,473
2,1,465


In [340]:
# fixing sample imbalance
rus = RandomUnderSampler(random_state=42)
data_df_res, y_res = rus.fit_resample(data1, data1['labels'])

In [341]:
# Splitting the dataset into the Training set and Test set (since we have a new output variable)
X_train, X_test, y_train, y_test = train_test_split(data_df_res['Author Keywords'], y_res, test_size = 0.2, random_state = 42)

In [342]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
cls = LogisticRegression()

pipe = make_pipeline(tfidf, cls)

In [343]:
pipe.fit(X_train,y_train) # fit model

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [344]:
# evaluate model performance on training set

y_eval = pipe.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.97      0.99       364
           1       0.97      0.98      0.97       379
           2       0.97      0.98      0.97       373

    accuracy                           0.98      1116
   macro avg       0.98      0.98      0.98      1116
weighted avg       0.98      0.98      0.98      1116



In [345]:
y_pred = pipe.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.88      0.68      0.77       101
           1       0.69      0.80      0.74        86
           2       0.77      0.85      0.81        92

    accuracy                           0.77       279
   macro avg       0.78      0.78      0.77       279
weighted avg       0.79      0.77      0.77       279



In [346]:
# run single prediction
# 'I am serious in asserting that my breath was entirely gone.' Edgar Allan Poe

t1 = ['Business, energy crisis and the world']

In [347]:
# preprocess

t1_p = text_prepro(pd.Series(t1)) # note, we need to pack text up as pd.Series 

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
100%|██████████| 1/1 [00:00<00:00, 82.92it/s]


In [348]:
# predict

pipe.predict(t1_p)

array([2])

In [349]:
# overall weights (works only for linear models)
eli5.show_weights(pipe, top=10, target_names=['0', '1', '2'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Network Analysys (NW)

## Data preparation

In [351]:
data = pd.read_csv("https://raw.githubusercontent.com/Alphambarushimana/Sustainability/main/scopuswithlabels.csv")
data

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Affiliations,Authors with affiliations,Abstract,Author Keywords,Index Keywords,Document Type,Publication Stage,Open Access,Source,EID
0,"Chapwanya M., Lubuma J., Terefe Y., Tsanou B.",26026788700;6602950766;54783401400;25823549000;,Analysis of War and Conflict Effect on the Tra...,2022,Bulletin of Mathematical Biology,84,12,136,NaN,NaN,...,Department of Mathematics and Applied Mathemat...,"Chapwanya, M., Department of Mathematics and A...",The tenth Ebola outbreak in the Democratic Rep...,Basic reproduction number; Conflict dynamics; ...,basic reproduction number; biological model; D...,Article,Final,NaN,Scopus,2-s2.0-85140065004
1,"Piat M., Wainwright M., Rivest M.-P., Sofouli ...",6604070039;35090631400;56871264100;18635052000...,The impacts of implementing recovery innovatio...,2022,International Journal of Mental Health Systems,16,1,49,NaN,NaN,...,"Department of Psychiatry, McGill University an...","Piat, M., Department of Psychiatry, McGill Uni...",Background: Implementing mental health recover...,Conceptual framework; Guidelines; Impact; Impl...,adult; article; conceptual framework; content ...,Article,Final,NaN,Scopus,2-s2.0-85139748940
2,"Velayati F., Ayatollahi H., Hemmat M., Dehghan R.",57219486899;55991128900;57170342800;57365458900;,The 4P telehealth business framework for Iran,2022,BMC Medical Informatics and Decision Making,22,1,266,NaN,NaN,...,"Telemedicine Research Center, National Researc...","Velayati, F., Telemedicine Research Center, Na...",Background: Telehealth services can utilize va...,Business; Health service marketing; Telehealth...,adult; article; comparative effectiveness; Del...,Article,Final,NaN,Scopus,2-s2.0-85139435002
3,"Pastor A.V., Nunes J.P., Ciampalini R., Bahri ...",55927118200;7102878032;6506995140;7004276837;5...,ScenaLand: a simple methodology for developing...,2022,Mitigation and Adaptation Strategies for Globa...,27,8,52,NaN,NaN,...,"LISAH, University Montpellier, INRAE, IRD, Ins...","Pastor, A.V., LISAH, University Montpellier, I...",Scenarios serve science by testing the sensiti...,Experts; Land degradation; Land use and manage...,agricultural soil; environmental protection; e...,Article,Final,NaN,Scopus,2-s2.0-85138219727
4,"Abdullah H.M., Dakurah T., Akoto H., Abaidoo B...",57887451300;12140148900;57213456448;5623694480...,Predictors of recurrence of chronic subdural h...,2022,BMC Neurology,22,1,346,NaN,NaN,...,"Department of Surgery, Korle Bu Teaching Hospi...","Abdullah, H.M., Department of Surgery, Korle B...",Background: Chronic subdural haematoma (CSDH) ...,Association; Chronic; Haematoma; Predictors; R...,anticoagulant agent; antithrombocytic agent; w...,Article,Final,NaN,Scopus,2-s2.0-85137878264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,"Armeanu D.S., Vintilă G., Gherghina S.C.",26434149400;26434536500;56046530600;,Empirical study towards the drivers of sustain...,2018,Sustainability (Switzerland),10,1,4,NaN,NaN,...,"Department of Finance, The Bucharest Universit...","Armeanu, D.S., Department of Finance, The Buch...",This study aims at empirically investigating t...,Economic growth; EU-28; Fixed and random effec...,economic growth; employment; European Union; h...,Article,Final,NaN,Scopus,2-s2.0-85038843701
1630,"Abaunza F., Hameri A.-P., Niemi T.",56060899300;7003735163;57215042589;,EEUI: a new measure to monitor and manage ener...,2018,International Journal of Productivity and Perf...,67,1,NaN,111,127,...,"Faculty of Business and Economics, University ...","Abaunza, F., Faculty of Business and Economics...",Purpose: Data centers (DCs) are similar to tra...,Data centres; Energy efficiency; Energy propor...,NaN,Article,Final,NaN,Scopus,2-s2.0-85038831501
1631,"Zhang Y., Lin Z.",57840767400;54894812400;,Predicting the helpfulness of online product r...,2018,Electronic Commerce Research and Applications,27,NaN,NaN,1,10,...,"School of

In [352]:
data_select = data[['Authors', 'Author(s) ID','EID']]

## Creating the edges

In [353]:
edges = pd.merge(data_select, data_select, on='EID')
edges.head()

,Authors_x,Author(s) ID_x,EID,Authors_y,Author(s) ID_y
0,"Chapwanya M., Lubuma J., Terefe Y., Tsanou B.",26026788700;6602950766;54783401400;25823549000;,2-s2.0-85140065004,"Chapwanya M., Lubuma J., Terefe Y., Tsanou B.",26026788700;6602950766;54783401400;25823549000;
1,"Piat M., Wainwright M., Rivest M.-P., Sofouli ...",6604070039;35090631400;56871264100;18635052000...,2-s2.0-85139748940,"Piat M., Wainwright M., Rivest M.-P., Sofouli ...",6604070039;35090631400;56871264100;18635052000...
2,"Velayati F., Ayatollahi H., Hemmat M., Dehghan R.",57219486899;55991128900;57170342800;57365458900;,2-s2.0-85139435002,"Velayati F., Ayatollahi H., Hemmat M., Dehghan R.",57219486899;55991128900;57170342800;57365458900;
3,"Pastor A.V., Nunes J.P., Ciampalini R., Bahri ...",55927118200;7102878032;6506995140;7004276837;5...,2-s2.0-85138219727,"Pastor A.V., Nunes J.P., Ciampalini R., Bahri ...",55927118200;7102878032;6506995140;7004276837;5...
4,"Abdullah H.M., Dakurah T., Akoto H., Abaidoo B...",57887451300;12140148900;57213456448;5623694480...,2-s2.0-85137878264,"Abdullah H.M., Dakurah T., Akoto H., Abaidoo B...",57887451300;12140148900;57213456448;5623694480...


In [354]:
edges = edges[edges.Authors_x	 != edges.Authors_y]

In [355]:
edges = edges.groupby(['Authors_x', 'Authors_y']).size().reset_index()

In [356]:
edges = []



for i in data.iterrows():
  targets = i[1]['Authors'].replace(' ', '').split(',')
  edges_i = [(i[1]['EID'], j) for j in targets]
  edges.extend(edges_i)

In [357]:
len(edges)

5757

In [358]:
edges[1]

('2-s2.0-85140065004', 'LubumaJ.')

In [359]:
G = nx.Graph()

In [360]:
G.add_edges_from(edges)

In [361]:
from community import community_louvain

In [362]:
centrality_dgr = nx.degree_centrality(G)
centrality_eig = nx.eigenvector_centrality_numpy(G)

partition = community_louvain.best_partition(G)

In [363]:
nx.set_node_attributes(G, centrality_dgr, 'dgr')
nx.set_node_attributes(G, centrality_eig, 'eig')
nx.set_node_attributes(G, partition, 'partition')

## Defining the nodes

In [364]:
nodes_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [365]:
nodes_df.sort_values('eig',ascending=False)[:50]

,dgr,eig,partition
2-s2.0-85124238268,0.007914,0.704520,90
Martinez-MillanaA.,0.000288,0.100429,90
GattaR.,0.000288,0.100429,90
Fernandez-LlatasC.,0.000288,0.100429,90
WeskeM.,0.000432,0.098364,90
StefaniniA.,0.000288,0.098292,90
AloiniD.,0.000288,0.098292,90
CapurroD.,0.000288,0.096556,90
Rinderle-MaS.,0.000288,0.096484,90
DiCiccioC.,0.000288,0.096450,90


In [366]:
nodes_df.partition.nunique()

1312

In [367]:
nodes_df.partition.value_counts()[:20]

80      150
134     127
90       79
104      66
36       52
39       46
67       44
118      42
494      41
371      36
1017     31
112      31
1093     28
55       28
438      26
248      25
402      24
212      23
177      23
31       23
Name: partition, dtype: int64

In [368]:
top10_com = nodes_df.partition.value_counts()[:10].index

In [369]:
# Complicated approach using NetworkX
# Who are the people in these?
top10_com_nodes = nodes_df[nodes_df.partition.isin(top10_com)].index

# Let's make a subgraph with only these people (one could also do it in pandas)
g_sub = nx.subgraph(G, top10_com_nodes)

# From here to dataframe

In [370]:
nodes_df_top10 = nodes_df[nodes_df.partition.isin(top10_com)]

In [371]:
nodes_df_top10

,dgr,eig,partition
2-s2.0-85138125994,0.000863,4.163996e-18,36
HuangY.,0.000432,5.503555e-18,36
BurgoineT.,0.000144,1.336909e-18,36
EssmanM.,0.000144,4.605822e-19,36
TheisD.R.Z.,0.000144,2.390555e-18,36
...,...,...,...
HendersonH.,0.000144,1.642954e-18,80
2-s2.0-85053941664,0.000144,-1.371592e-17,39
2-s2.0-85048985450,0.000144,-1.572162e-17,39
2-s2.0-85044453860,0.000144,-1.566741e-17,39


In [372]:
top_people = nodes_df_top10.groupby('partition')['eig'].nlargest(5).reset_index()

In [373]:
top_people

,partition,level_1,eig
0,36,2-s2.0-85129964537,9.861620e-18
1,36,2-s2.0-85128484169,7.633025e-18
2,36,HuangY.,5.503555e-18
3,36,LiS.,4.880173e-18
4,36,ShiQ.,4.286781e-18
5,39,[Noauthornameavailable],-4.162312e-18
6,39,2-s2.0-85126360488,-1.149338e-17
7,39,2-s2.0-85098630721,-1.154759e-17
8,39,2-s2.0-85088956973,-1.165600e-17
9,39,2-s2.0-85133222920,-1.257754e-17


## Visualization

In [374]:
!pip install -qq holoviews
!pip install -qq -U bokeh
!pip install -qq datashader

     |████████████████████████████████| 18.5 MB 306 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.12.1 requires bokeh<2.4.0,>=2.3.0, but you have bokeh 2.4.3 which is incompatible.
     |████████████████████████████████| 18.2 MB 617 kB/s 
     |████████████████████████████████| 76 kB 5.0 MB/s 


In [376]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.plotting import show
kwargs = dict(width=800, height=800, xaxis=None, yaxis=None)
opts.defaults(opts.Nodes(**kwargs), opts.Graph(**kwargs))

ImportError: ignored

In [ ]:
top_central_nodes = nodes_df[nodes_df.eig > nodes_df.eig.quantile(0.99)].index

In [ ]:
g_sub = nx.subgraph(G, top_central_nodes)

In [ ]:
# Create and save a layout.
g_layout = nx.layout.spring_layout(g_sub) 
g_plot = hv.Graph.from_networkx(g_sub, g_layout).opts(tools=['hover'], node_color='partition')

## Insert names again into the dataset for labels <3
labels = hv.Labels(g_plot.nodes, ['x', 'y'], 'index')

In [ ]:
# make the plot
from holoviews.operation.datashader import datashade, bundle_graph
bundled = bundle_graph(g_plot)

In [ ]:
show(hv.render(bundled * labels.opts(text_font_size='6pt', text_color='white', bgcolor='gray')))